In [50]:
%matplotlib
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
from PIL import Image
from scipy import ndimage
from scipy import misc

Using matplotlib backend: MacOSX


In [2]:
IMAGE_FOLDER_PATH = "/Users/zackakil/Desktop/capture clean/"

In [3]:
image_labels = pd.Series.from_csv('rugby_image_labels.csv')
image_labels = image_labels.dropna()

In [4]:
image_labels.keys()[0]

'1489359202.14.jpg'

In [5]:
output = {}

face = misc.imread(IMAGE_FOLDER_PATH+image_labels.keys()[0], flatten=True)
face1 = misc.imread(IMAGE_FOLDER_PATH+image_labels.keys()[1], flatten=True)     
plt.imshow((face[200:300] - face1[200:300])>30)
plt.show()

In [32]:
UPPER_FRAME = 300
LOWER_FRAME = 200
THRESHOLD = 30
def comapare_frames(frame_1_file_name, frame_2_file_name):
    f1 = misc.imread(frame_1_file_name, flatten=True)
    f2 = misc.imread(frame_2_file_name, flatten=True)
    diff = (f1[LOWER_FRAME:UPPER_FRAME] - f2[LOWER_FRAME:UPPER_FRAME])>THRESHOLD
    return [diff.sum(axis=0),diff]

def get_image(file_name):
    return misc.imread(file_name)

In [33]:
frame_compares_sum = []
frame_compares = []
frame_images = []

images_to_process = image_labels[:1000]
for index, value in images_to_process[:-1].iteritems():
    curr_image = IMAGE_FOLDER_PATH + index
    next_image_index = image_labels.index.get_loc(index)+1
    next_image_key = IMAGE_FOLDER_PATH + image_labels.keys()[next_image_index]
    out = comapare_frames(curr_image, next_image_key)
    frame_compares.append(out[1])
    frame_compares_sum.append(out[0])
    frame_images.append(get_image(next_image_key))

In [34]:
"""
A simple example of an animated plot
"""
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

fig, ax = plt.subplots()

x = np.arange(0, 2*np.pi, 0.01)        # x-array
line, = ax.plot(frame_compares_sum[0])

def animate(i):
    line.set_ydata(frame_compares_sum[i])  # update the data
    return line,

#Init only required for blitting to give a clean slate.
def init():
    line.set_ydata(frame_compares_sum[0])
    return line,

ani = animation.FuncAnimation(fig, animate, np.arange(1, 99), init_func=init,
    interval=100, blit=False)
plt.show()

In [106]:
fig, (ax1, ax2, ax3) = plt.subplots(3,1)

im = ax1.imshow(frame_images[0][LOWER_FRAME:UPPER_FRAME], animated=True)
im2 = ax2.imshow(frame_compares[0], animated=True)
im3, = ax3.plot(frame_compares_sum[0])

line = [im, im2, im3]

def updatefig(i):
    line[0].set_array(frame_images[i][LOWER_FRAME:UPPER_FRAME])
    line[1].set_array(frame_compares[i])
    line[2].set_ydata(frame_compares_sum[i])   
    return line

ani = animation.FuncAnimation(fig, 
                              updatefig,
                              np.arange(200, 300),
                              interval=150,
                              blit=True)
plt.show()

In [52]:
ani.save('animation.gif', writer='imagemagick', fps=10)